In [1]:
// #region seen

enum ValueType {
    Int,
    Bool,
    Function,
    FunctionNative,
}
type Value =
    | { type: ValueType.Int, value: number }
    | { type: ValueType.Bool, value: boolean }
    | { type: ValueType.Function, body: Expr, binding: Ident, context: Context }

type Context = Record<Ident, Value>

function evaluate(expr: Expr, context: Context = {}): Value {
    switch (expr.type) {
        case ExprType.Application: {
            const lambda = evaluate(expr.lambda, context)
            const argument = evaluate(expr.argument, context)
            if (lambda.type === ValueType.Function) {
                return evaluate(lambda.body, { ...lambda.context, [lambda.binding]: argument })
            }
            throw new Error("Runtime eval error: cannot call type " + ValueType[lambda.type])
        }
        case ExprType.Abstraction: {
            const binding = expr.binding.name
            const body = expr.body
            return {
                type: ValueType.Function,
                binding,
                body,
                context,
            }
        }
        case ExprType.Var: {
            const value = context[expr.name]
            if (value == undefined) {
                throw new Error("Runtime eval error: variable `" + expr.name + "` not found")
            }
            return value
        }
        case ExprType.LiteralInt: {
            return { type: ValueType.Int, value: expr.value }
        }
        case ExprType.LiteralBool: {
            return { type: ValueType.Bool, value: expr.value }
        }
    }
}

//#endregion

In [2]:
enum TypeType {
    Int,
    Bool,
    Function,
}
type Type =
    | { type: TypeType.Int }
    | { type: TypeType.Bool }
    | { type: TypeType.Function, argumentType: Type, returnType: Type }

//#region seen
enum ExprType {
    Application,
    Abstraction,
    Var,
    LiteralInt,
    LiteralBool,
}
type Expr =
    | { type: ExprType.Application, lambda: Expr, argument: Expr }
    | { type: ExprType.Abstraction, binding: Binding, body: Expr }
    | { type: ExprType.Var, name: Ident }
    | { type: ExprType.LiteralInt, value: number }
    | { type: ExprType.LiteralBool, value: boolean }

type Ident = string
type Binding = { name: Ident, type: Type }
//#endregion

In [4]:
//#region seen
import { Runtime } from 'jsr:@kawcco/parsebox'
import { OurModule } from "../grammar.ts"

const { Const, Tuple, Union, Ident, Ref, Array, Optional } = Runtime

const Tokens = {
    Arrow: Const('->'),
    LParen: Const('('),
    RParen: Const(')'),
    LBracket: Const('['),
    RBracket: Const(']'),
    Int: Const('int'),
    Bool: Const('bool'),
    Fn: Const('fn'),
    Comma: Const(','),
    Colon: Const(':'),
    Let: Const('let'),
    In: Const('in'),
    Equals: Const('='),
    If: Const('if'),
    Then: Const('then'),
    Else: Const('else'),
    True: Const('true'),
    False: Const('false'),
}
//#endregion

const Language = new OurModule({
    Ty: Union(
        [
            Tokens.Int,
            Tokens.Bool,
            Ref<Type>('TyFn'),
        ],
        raw => (
            raw == "int"
                ? { type: TypeType.Int }
                : raw == "bool"
                    ? { type: TypeType.Bool }
                    : raw
        ) satisfies Type,
    ),
    TyFn: Tuple(
        [
            Tokens.Fn,
            Tokens.LBracket,
            Ref<Type>('Ty'),
            Tokens.Comma,
            Ref<Type>('Ty'),
            Tokens.RBracket,
        ],
        ([, , argumentType, , returnType]) =>
            ({ type: TypeType.Function, argumentType, returnType } satisfies Type),
    ),
    Binding: Tuple(
        [
            Ident(),
            Tokens.Colon,
            Ref<Type>('Ty'),
        ],
        ([name, , type]) => ({ name, type } as Binding),
    ),
    //#region seen
    Expr: Tuple(
        [
            Ref<Expr>('ExprWithoutApplication'),
            Array(Tuple([
                Tokens.LParen,
                Ref<Expr>('Expr'),
                Tokens.RParen,
            ], ([, expr,]) => expr)),
        ],
        ([base, applicationArgs]) => {
            let expr = base
            for (const argument of applicationArgs) {
                expr = { type: ExprType.Application, lambda: expr, argument }
            }
            return expr
        }
    ),
    ExprWithoutApplication: Union(
        [
            Ref<Expr>('Abstraction'),
            Ref<Expr>('Int'),
            Ref<Expr>('Bool'),
            Ref<Expr>('Var'),
            Ref<Expr>('ExprParen'),
        ]
    ),
    ExprParen: Tuple(
        [
            Tokens.LParen,
            Ref<Expr>('Expr'),
            Tokens.RParen,
        ],
        ([, expr,]) => expr
    ),
    Abstraction: Tuple(
        [
            Ref<Binding>('Binding'),
            Tokens.Arrow,
            Ref<Expr>('Expr')
        ],
        ([binding, , body]) => ({ type: ExprType.Abstraction, binding, body } satisfies Expr),
    ),
    Var: Tuple(
        [
            Ident(),
        ],
        ([name]) => ({ type: ExprType.Var, name } satisfies Expr),
    ),
    Digit: Union([
        Const("0"),
        Const("1"),
        Const("2"),
        Const("3"),
        Const("4"),
        Const("5"),
        Const("6"),
        Const("7"),
        Const("8"),
        Const("9"),
    ]),
    Int: Tuple(
        [
            Optional(Const("-")),
            Ref("Digit"),
            Array(Ref("Digit"))
        ],
        ([[minus], first_digit, digits]) => (
            {
                type: ExprType.LiteralInt,
                value: parseInt((minus ?? "") + first_digit + digits.join(""))
            } satisfies Expr
        ),
    ),
    Bool: Union(
        [
            Tokens.True,
            Tokens.False,
        ],
        raw => ({ type: ExprType.LiteralBool, value: raw == "true" } satisfies Expr),
    ),
    //#endregion
})


In [5]:
import { inspectType } from "../inspect.ts"

function solveTypes(expr: Expr, context: Record<string, Type> = {}): Type {
    switch (expr.type) {
        /// @impl
        case ExprType.Application: {
            const argumentType = solveTypes(expr.argument, context)
            const lambdaType = solveTypes(expr.lambda, context)
            if (lambdaType.type !== TypeType.Function) {
                throw new Error(`solveTypes: cannot call value of type ${inspectType(lambdaType)}`)
            }
            if (!typesMatch(lambdaType.argumentType, argumentType)) {
                throw new Error(`solveTypes: lambda argument type mismatched: expected type ${inspectType(lambdaType.argumentType)}, found ${inspectType(argumentType)}`)
            }
            return lambdaType.returnType
        }
        /// @impl
        case ExprType.Abstraction: {
            return {
                type: TypeType.Function,
                argumentType: expr.binding.type,
                returnType: solveTypes(expr.body, {
                    ...context,
                    [expr.binding.name]: expr.binding.type,
                })
            }
        }
        case ExprType.Var: {
            if (expr.name in context) {
                return context[expr.name]
            } else {
                throw new Error(`solveTypes: variable '${expr.name}' not found`)
            }
        }
        case ExprType.LiteralInt:
            return { type: TypeType.Int }
        case ExprType.LiteralBool:
            return { type: TypeType.Bool }
    }
}

function typesMatch(a: Type, b: Type): boolean {
    switch (a.type) {
        case TypeType.Int:
            return b.type === TypeType.Int
        case TypeType.Bool:
            return b.type === TypeType.Bool
        /// @impl
        case TypeType.Function: {
            if (b.type !== TypeType.Function) {
                return false
            }
            return typesMatch(a.argumentType, b.argumentType)
                && typesMatch(a.returnType, b.returnType)
        }
    }
}

In [ ]:
import { describe, it } from "jsr:@std/testing/bdd"
import { expect } from "jsr:@std/expect"

describe("integration test of parse-typecheck", () => {
    const parse = (code: string) => Language.Parse("Expr", code) as [Expr, string] | []
    const type = (code: string) => (Language.Parse("Ty", code) as [Type, string] | [])[0] ?? (() => { throw new Error() })()
    const parse_typecheck = (code: string) => {
        const parsed = parse(code)
        return parsed.length === 2 ? solveTypes(parsed[0]) : null
    }
    // @impl
    it("can solve types of simple expressions", () => {
        expect(parse_typecheck("1234")).toStrictEqual(type("int"))
        expect(parse_typecheck("true")).toStrictEqual(type("bool"))
        expect(parse_typecheck("odd")).toStrictEqual(type("fn[int, bool]"))
        expect(parse_typecheck("x: int -> true")).toStrictEqual(type("fn[int, bool]"))
        expect(parse_typecheck("x: int -> x")).toStrictEqual(type("fn[int, int]"))
        expect(() => parse_typecheck("x: int -> y")).toThrow()
    })
})

In [7]:
//////////////////////////////////// REPL TIME ///////////////////////////////////////
import { repl } from "../repl.ts"
await repl(solveTypes as never, Language, evaluate as never)